# Build Segment Library

With hydro-conditioned DEM, we will identify streams, divide them into segments and create a library for each segment.

## Import python packages

In [13]:
import os

# # flnpln library
# from fldpln_header import *
# from fldpln_library import *

import fldpln_py
# Import the matlab module only after you have imported MATLAB Compiler SDK generated Python modules.
import matlab

## Initialize the FLDPLN Python Library

Before we can use the FLDPLN model, we must first initialize it. This will activate the MATLAB Runtime and will take a littlle bit of time.

In [14]:
try:
    my_fldpln_py = fldpln_py.initialize()
except Exception as e:
    print('Error initializing fldpln_py package\\n:{}'.format(e))
    exit(1)

## Generate Segments

The first step to create a FLDPLN library is to identify stream networks and create stream segments from a hydro-conditioned digital elevation model (DEM). The stream networks/pixels are first identified using a flow accumulation threshold ('strthr'). The stream networks are first divided into reaches (stream links in ArcGIS term), which are stream segments between headwater and confluence pixels or between two consecutive confluence pixels. Note that pixels flow out of the DEM are treated as confluence pixels in this process. Those natural reaches/segments are further divided based on flow accumulation jumps greater than or equal to 'facthr'. Then the segments are bisected until all segments are no longer than 'seglen'.

The default values for the 3 parameters are: 70 sq. miles, 25 sq. miles, and 2 miles for the libraries in KS. Note that Kansas FLDPLN libraries stream networks are more coarse grained than the National Water Model (NWM) reaches. This is because the stream networks are generated just to use available USGS gauges. If the NWM discharge is used for the flood inundation mapping, the stream networks need to be densified.

The FSP and segment information are also saved as two CSV files for generating segment shapefile for selecting a subset of segments for build library.

In [18]:
try:
    fdrfIn = "E:/CUAHSI_SI/VerdigrisRv/projects/verdigris_10m/bil/fdr.bil"
    facfIn = "E:/CUAHSI_SI/VerdigrisRv/projects/verdigris_10m/bil/fac.bil"
    strfacIn = matlab.double([70], size=(1, 1))
    segfacIn = matlab.double([25], size=(1, 1))
    seglenIn = matlab.double([2], size=(1, 1))
    segdirIn = "E:/CUAHSI_SI/VerdigrisRv/projects/verdigris_10m/segs_py"
    my_fldpln_py.create_segments(fdrfIn, facfIn, strfacIn, segfacIn, seglenIn, segdirIn, nargout=0)

    # write FSP and segment info CSV files
    bildirIn = "E:/CUAHSI_SI/VerdigrisRv/projects/verdigris_10m/bil"
    segdirIn = "E:/CUAHSI_SI/VerdigrisRv/projects/verdigris_10m/segs_py"
    seg_listIn = matlab.double([], size=(0, 0)) # empty seg_list will use all the segment
    # outdirIn = "E:/fldpln/sites/wildcat_10m_3dep/segs"
    my_fldpln_py.write_fsp_segment_info_files(bildirIn, segdirIn, seg_listIn, segdirIn, nargout=0)
    
except Exception as e:
    print('Error occurred during program execution\\n:{}'.format(e))

## Generate Segment Shapefile 

Shapefiles can be created using the FSP and segment info CSV file. The shapefile can then be used in a GIS software to examine and select a sub set of segments to create their libraries. Note that the shapefile gets its CRS from segment BIL raster's .prj file and has all the attributes in the FSP and segment info CSV file.

Let's first import necessary modules

In [8]:
# flnpln library
from fldpln_header import *
from fldpln_library import *

ModuleNotFoundError: No module named 'fldpln_header'

In [9]:
# segment BIL folder
segFolder= r'E:/fldpln/sites/wildcat_10m_3dep/segs_py'

# FSP and segment info CSV files
segInfoFile = os.path.join(segFolder,segInfoFileName)
fspInfoFile = os.path.join(segFolder,fspInfoFileName)

# shapefile name
shpName = "segments.shp"
outShpFile =  os.path.join(segFolder,shpName)

# Read CRS from .prj file
# read in spatial reference for the library
srFile = os.path.join(segFolder,'str_segid.prj')
with open(srFile, 'r') as srf:
    srText = srf.read()
libCrs = CRS.from_wkt(srText)

# generate segment shapefile 
print(f'Generate segment shapefile for libraries ...')
GenerateSegmentShapefilesFromFspSegmentInfoFiles(segInfoFile, fspInfoFile, libCrs, outShpFile)

NameError: name 'segInfoFileName' is not defined

Segment shapefile can also be created using the ArcGIS Pro's Stream to Features tool. Note that the Raster to Polyline tool doesn't work properly in some cases. In the figure below, the left stream polyline is generated by the Raster to Polyline tool, and the right stream polyline is generated by the Stream to Feature tool. The left polyline is erroneous. Also note that the Stream to Features tool connects upstream segments at the downstream confluence pixel.

![](./stream2feature.PNG)

## Examine and Select Segments

Using the shapefiles in a GIS, we can take a close look on the segments. We may delete some segments in large waterbodies (e.g., lakes) and just select a subset of segment to build their library. In ArcGIS Pro, we can simply export the selected segments as a CSV file. This file can be used as one of the inputs to build segment library. Make sure that the first column is the segment ID.

## Create Segment Library

With the segments identified, we can create the library for each of them or a subset of selected segments. **Note that this is the most time consuming step in building FLDPLN libraries, especially the max stage modelded is high.**

### Create a library for selected segments

With the segment shapefiles, we can select the subset of segments for the Wildcat Creek (sgement IDs from 1 to 16) to create a library. The selected segments can be exported as CSV file (wildcat_creek_segments.csv) in GIS. The first column in the file is assumed as segment IDs. Note that the CSV has a header row.

All the selected segments have the same max. flood stage as specified by mxht. We can also add a second column to the CSV file to specify different max. flood stages for different segment.

In [10]:
try:
    bildirIn = "E:/CUAHSI_SI/training/examples/wildcat_10m_3dep/bil"
    segdirIn = "E:/CUAHSI_SI/training/examples/wildcat_10m_3dep/segs_py"
    segfileIn = "E:/CUAHSI_SI/training/examples/wildcat_10m_3dep/segs_py/segment_clip.csv" #Important to generate this .csv
    fldmnIn = matlab.double([0.01], size=(1, 1))
    fldmxIn = matlab.double([12], size=(1, 1)) #THIS IS THE MOST IMPORTANT PARAMETER FOR DEFINING FLOOD ELEVATION CONDITIONS
    dhIn = matlab.double([1], size=(1, 1))
    mxhtIn = matlab.double([0], size=(1, 1)) 
    libdirIn = "E:/CUAHSI_SI/training/examples/wildcat_10m_3dep/seglib_py"
    mtypeIn = "ram"
    paraIn = {"type": "parfeval"}
    my_fldpln_py.create_segment_library_v6(bildirIn, segdirIn, segfileIn, fldmnIn, fldmxIn, dhIn, mxhtIn, libdirIn, mtypeIn, paraIn, nargout=0)
except Exception as e:
    print('Error occurred during program execution\\n:{}'.format(e))

### Create a library for all the segments

We can set the segment file (segfileIn) as empty (i.e., "") and the library will be created using the maximum stage for all the segments.

In [ ]:
try:
    bildirIn = "E:/fldpln/sites/wildcat_10m_3dep/bil"
    segdirIn = "E:/fldpln/sites/wildcat_10m_3dep/segs_py"
    segfileIn = ""
    fldmnIn = matlab.double([0.01], size=(1, 1))
    fldmxIn = matlab.double([15], size=(1, 1))
    dhIn = matlab.double([1], size=(1, 1))
    mxhtIn = matlab.double([0], size=(1, 1))
    libdirIn = "E:/fldpln/sites/wildcat_10m_3dep/seglib_all_py"
    mtypeIn = "ram"
    paraIn = {"type": "parfeval"}
    my_fldpln_py.create_segment_library_v6(bildirIn, segdirIn, segfileIn, fldmnIn, fldmxIn, dhIn, mxhtIn, libdirIn, mtypeIn, paraIn, nargout=0)
except Exception as e:
    print('Error occurred during program execution\\n:{}'.format(e))

### Work with Existing Library

If segment libraies already exist and if the specified fldmx is lower than the existing library’s max. stage, the model simply generates the new library with existing information and DOES NOT re-run the model. 

If the specified fldmx is higher than the existing library’s max. stage, the model continues from the existing library without starting from the scratch.

Note that the model creates the new segment libraries  under the SAME folder where the existing libraries reside.

In [ ]:
try:
    bildirIn = "E:/fldpln/sites/wildcat_10m_3dep/bil"
    segdirIn = "E:/fldpln/sites/wildcat_10m_3dep/segs_py"
    segfileIn = "E:/fldpln/sites/wildcat_10m_3dep/segs_py/sub_segs_fldmxs_changed.xlsx" # some segments have higher fldmx and others have lower fldmx than the existing libraries
    fldmnIn = matlab.double([0.01], size=(1, 1))
    fldmxIn = matlab.double([15], size=(1, 1))
    dhIn = matlab.double([1], size=(1, 1))
    mxhtIn = matlab.double([0], size=(1, 1))
    libdirIn = "E:/fldpln/sites/wildcat_10m/seglib_test_py"
    mtypeIn = "ram"
    paraIn = {"type": "parfeval"}
    my_fldpln_py.create_segment_library_v6(bildirIn, segdirIn, segfileIn, fldmnIn, fldmxIn, dhIn, mxhtIn, libdirIn, mtypeIn, paraIn, nargout=0)
except Exception as e:
    print('Error occurred during program execution\\n:{}'.format(e))

## Format Segment Library for Tiling

In [11]:
try:
    bildirIn = "E:/CUAHSI_SI/training/examples/wildcat_10m_3dep/bil"
    segdirIn = "E:/CUAHSI_SI/training/examples/wildcat_10m_3dep/segs_py"
    libdirIn = "E:/CUAHSI_SI/training/examples/wildcat_10m_3dep/seglib_py"
    diroutIn = "E:/CUAHSI_SI/training/examples/wildcat_10m_3dep/lib_py"
    my_fldpln_py.format_segment_library(bildirIn, segdirIn, libdirIn, diroutIn, nargout=0)
except Exception as e:
    print('Error occurred during program execution\\n:{}'.format(e))

## Terminate the FLDPLN library

In [12]:
my_fldpln_py.terminate()

## Issues and Questions

* Messages printed out (disp() or fprintf()) to the stdout in the FLDPLN Python package CANNOT be printed in this notebook. Running python scripts (such as wildcat_segments.py, wildcat_library.py and wildcat_format_library.py) works just fine.
